# 64x64 DCGAN experiment

## Importy + mount drive


In [1]:
# imports
import sys
if 'google.colab' in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')
  output_path = "/content/drive/My Drive/gan_files"
  %rm -r /content/BP-Bielko-GAN-Docker
  %rm -r /content/src
  !git clone https://github.com/2021-FIIT-Bc-projects/BP-Bielko-GAN-Docker.git
  %cd BP-Bielko-GAN-Docker/gan
else:
    output_path = ""

from src.dcgan_models import *

Mounted at /content/drive
rm: cannot remove '/content/BP-Bielko-GAN-Docker': No such file or directory
rm: cannot remove '/content/src': No such file or directory
Cloning into 'BP-Bielko-GAN-Docker'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 139 (delta 55), reused 101 (delta 28), pack-reused 0
Receiving objects: 100% (139/139), 138.16 MiB | 29.43 MiB/s, done.
Resolving deltas: 100% (55/55), done.
cp: cannot stat '/content/BP-Bielko-GAN-Docker/src': No such file or directory
/content/BP-Bielko-GAN-Docker/gan


In [ ]:
list_physical_devices('GPU')

NameError: name 'list_physical_devices' is not defined

## Stiahnutie Flickr-Faces-HQ datasetu
Možnosť A: Stiahnuť dataset nanovo - veľká pravdepodobnosť vyčerpania download kvóty

Možnosť B: Načítať pred-pripravený zip súbor s datasetom 

In [ ]:
# A
# %cd /content/
!git clone https://github.com/NVlabs/ffhq-dataset.git
%cd ffhq-dataset
!python download_ffhq.py --thumbs

Cloning into 'ffhq-dataset'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 67 (delta 2), reused 4 (delta 2), pack-reused 60
Unpacking objects: 100% (67/67), 3.88 MiB | 1.44 MiB/s, done.
/home/jovyan/gan/ffhq-dataset
\ 100.00% done  2/2 files  0.25/0.25 GB   3.58 MB/s   ETA: done    
Parsing JSON metadata...
\   2.36% done  1656/70001 files  0.05/1.96 GB   1.13 MB/s   ETA: 28m 48s ^C
Traceback (most recent call last):
  File "/home/jovyan/gan/ffhq-dataset/download_ffhq.py", line 445, in <module>
    run_cmdline(sys.argv)
  File "/home/jovyan/gan/ffhq-dataset/download_ffhq.py", line 440, in run_cmdline
    run(**vars(args))
  File "/home/jovyan/gan/ffhq-dataset/download_ffhq.py", line 409, in run
    download_files(specs, **download_kwargs)
  File "/home/jovyan/gan/ffhq-dataset/download_ffhq.py", line 204, in download_files
    exc_info = exception_queue.get(timeout=status_delay)
  File "/

In [2]:
# B
!mkdir /content/ffhq-dataset
!unzip "/content/drive/MyDrive/ffhq_dataset_thumbs/dataset_download.zip" -d "/content/ffhq-dataset"

Streaming output truncated to the last 5000 lines.
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65005.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65006.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65007.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65008.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65009.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65010.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65011.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65012.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65013.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65014.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65015.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65016.png  
  inflating: /content/ffhq-dataset/thumbnails128x128/65000/65017.png  
  inflating: /content/ffhq

## Definície modelov, funkcií - TODO real sample dict + predeterined samples prerob, sprav funkciu co nastavi rozmery

In [9]:
# definicie modelu

#default_width = 64
#default_height = 64
#default_dims = 100
#pixel_depth = 3 #rgb

model_name = "dcgan_128_test"

p_dims = 100
p_n = 100
predetermined_inputs = np.random.randn(p_dims * p_n)  # n vektorov z gauss. distrib.
predetermined_inputs = predetermined_inputs.reshape(p_n, p_dims)

## Vytvorenie modelu

In [11]:
height = 128
width = 128

generator = Generator(height, width, n_dim=100, n_paralell_samples=128, init_size=4)
discriminator = Discriminator(height, width, n_filters=128, dataset_path="dataset_download/thumbnails128x128")

gan = GAN(generator, discriminator, height=height, width=width, model_name=model_name, output_path=output_path)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## TODO tieto metadata ukladaj, zapisuj do metadata list index z batch namiesto času v hod.

In [ ]:
open(f'{output_path}/{model_name}/outputs/evaluation/epoch_metadata.txt', 'w').close()

In [6]:
metadata_list = []

## Trénovací proces

In [13]:
from_epoch = 2
to_epoch = 4

dataset_size = 70000

a = datetime.datetime.now()

gan.train_gan(dataset_size, metadata_list,
                n_dim=100, start_epoch=from_epoch, n_epochs=to_epoch,
                n_batch=dataset_size//700, n_eval=2, eval_samples=100, n_plot=2, plot_size=9)

b = datetime.datetime.now()
print("Time taken: ", b - a)

generator.model.save_weights(f"{output_path}/{model_type}/saves/generator_{to_epoch}.hdf5", overwrite=True)
discriminator.model.save_weights(f"{output_path}/{model_type}/saves/discriminator_{to_epoch}.hdf5", overwrite=True)

[Epoch 2] Batch 0/1400


FileNotFoundError: ignored

In [ ]:
print(generator.model.summary())
print(discriminator.model.summary())

In [12]:
load_epoch = 2
generator.model.load_weights(f"{output_path}/{model_name}/saves/generator_{2}.hdf5")
discriminator.model.load_weights(f"{output_path}/{model_name}/saves/discriminator_{2}.hdf5")